# Optimización del Activo
## Import de paquetes y funciones

In [1]:
library("readxl")
library("lubridate")
library('tidyr')
library('ggplot2')
library('YieldCurve')

Warning message:
"package 'readxl' was built under R version 3.6.3"Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union

Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'YieldCurve' was built under R version 3.6.3"Loading required package: xts
Loading required package: zoo
Warning message:
"package 'zoo' was built under R version 3.6.3"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 


In [2]:
search()

[1] ".GlobalEnv"         "package:YieldCurve" "package:xts"       
 [4] "package:zoo"        "package:ggplot2"    "package:tidyr"     
 [7] "package:lubridate"  "package:readxl"     "jupyter:irkernel"  
[10] "jupyter:irkernel"   "package:stats"      "package:graphics"  
[13] "package:grDevices"  "package:utils"      "package:datasets"  
[16] "package:methods"    "Autoloads"          "package:base"

In [3]:
source("helpers.R")

## Import de bases de datos de los indices
En esta sección se importan todos los datos de las tasas de interés de los bonos y activos requeridos.   
Adicionalmente, se encuentra la estimación de Nelson Sieguel para descomponer el comportamiento de las tasas.

In [4]:
path <- 'C:/Users/Diana C Contreras/OneDrive - Universidad de Los Andes/Riesgo Financiero/Talleres/T1-Riesgo/Taller 2/data/data.xlsx'

### Empezamos con los bonos TES en pesos.

In [5]:
tes_cop <- read_excel(path, sheet=1,skip=7, col_names=c('Date','6m','1y','3y','5y','10y','15y'))

In [6]:
tes_cop <- tes_cop[tes_cop$Date >= "2012-04-26",]
tes_cop <- as.data.frame(tes_cop)
rownames(tes_cop) <- tes_cop$Date
tes_cop <- tes_cop[,-c(1)]
tes_cop <- sapply(tes_cop, as.numeric)

head(tes_cop)


6m,1y,3y,5y,10y,15y
5.4042,5.5328,6.0056,6.4139,6.9276,7.3809
5.3703,5.5107,5.9953,6.4264,6.9410,7.3830
5.3749,5.5176,6.0044,6.4312,6.9430,7.3839
5.4176,5.5513,6.0613,6.4281,6.9308,7.3765
5.4304,5.5508,6.0495,6.4076,6.8896,7.3203
5.4570,5.5736,6.0911,6.4363,6.9042,7.3267


In [7]:
tes_maturity <- c(0.5,1,3,5,10,15)
tes_cop_ns <- Nelson.Siegel( rate=tes_cop,	maturity=tes_maturity)
head(tes_cop_ns)

beta_0,beta_1,beta_2,lambda
8.378015,-3.115947,0.03615750,0.1887807
8.308243,-3.090228,-0.07008414,0.2109684
8.295119,-3.071663,-0.04108135,0.2109684
8.253432,-2.979203,-0.04322311,0.2109684
8.161660,-2.874677,-0.02401636,0.2109684
8.095042,-2.787337,-0.02597386,0.2241687


### TES en UVR   
Notese que en este caso se importan directamente los resultados de Nelson Sieguel

In [8]:
tes_uvr_ns <- read_excel(path, sheet='TES UVR Betas',skip=1, col_names=c('Date','beta_0','beta_1','beta_2','lambda'))
tes_uvr_ns <- as.data.frame(tes_uvr_ns)
rownames(tes_uvr_ns) <- tes_uvr_ns$Date
tes_uvr_ns <- tes_uvr_ns[,-c(1)]
head(tes_uvr_ns)

,beta_0,beta_1,beta_2,lambda
,<dbl>,<dbl>,<dbl>,<dbl>
2022-04-08,0.09647801,-0.03119406,0.04511467,3.7
2022-04-07,0.09405698,-0.02969244,0.04226096,3.7
2022-04-06,0.09469592,-0.02990299,0.04248866,3.7
2022-04-05,0.09408631,-0.02883965,0.04186647,3.7
2022-04-04,0.09664698,-0.03390216,0.04312384,3.7
2022-04-01,0.09685086,-0.03217370,0.04468412,3.7


### Corporativo a EEUU

In [9]:
corp_usa <- read_excel(path, sheet='Corporativo A EEUU',skip=8, col_names=c('Date','3m','6m','1y','3y','5y','10y','15y'))

corp_usa <- as.data.frame(corp_usa)
rownames(corp_usa) <- corp_usa$Date
corp_usa <- corp_usa[,-c(1)]
head(corp_usa)

,3m,6m,1y,3y,5y,10y,15y
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2009-10-13,1.1404,1.2181,1.5046,2.5661,3.4811,4.7076,5.3309
2009-10-14,1.1381,1.2208,1.5196,2.6013,3.5293,4.7843,5.4388
2009-10-15,1.1055,1.1932,1.5041,2.6295,3.5733,4.8308,5.4545
2009-10-16,1.2114,1.2897,1.5619,2.6139,3.5447,4.7820,5.4008
2009-10-19,1.1778,1.2627,1.5420,2.5997,3.5283,4.7502,5.3610
2009-10-20,1.1593,1.2421,1.5096,2.5454,3.4739,4.7214,5.3203


In [10]:
corp_usa_maturity <- c(3/12,0.5,1,3,5,10,15)
corp_usa_ns <- Nelson.Siegel( rate=corp_usa,	maturity=corp_usa_maturity)
head(corp_usa_ns)

,beta_0,beta_1,beta_2,lambda
,<dbl>,<dbl>,<dbl>,<dbl>
2009-10-13,6.461855,-5.425669,-4.169706,0.5517879
2009-10-14,6.731600,-5.748756,-3.643443,0.4781896
2009-10-15,6.607364,-5.625489,-4.118086,0.5517879
2009-10-16,6.550958,-5.437508,-4.269783,0.5517879
2009-10-19,6.495381,-5.421099,-4.152560,0.5517879
2009-10-20,6.476177,-5.408359,-4.283447,0.5517879


### ETF's de seguimiento de la economía colombiana (COLCAP) y estadounidense (SPX)

In [11]:
index <- read_excel(path, sheet='Indices Accionarios',skip=7, col_names=c('Date','colcap','spx'))
index <- as.data.frame(index)
rownames(index) <- index$Date
index <- index[,-c(1)]
head(index)

,colcap,spx
,<dbl>,<dbl>
2009-10-13,1303.03,1073.19
2009-10-14,1294.39,1092.02
2009-10-15,1294.11,1096.56
2009-10-16,1301.69,1087.68
2009-10-19,1330.80,1097.91
2009-10-20,1315.97,1091.06


### Indice de Precios al consumidor

In [16]:
ipc <- read_excel(path, sheet='IPC',skip=1, col_names=c('Date','ipc'))
ipc <- as.data.frame(ipc)

head(ipc)

,Date,ipc
,<dttm>,<dbl>
1,1954-07-31,0.02632
2,1954-08-31,0.02612
3,1954-09-30,0.02576
4,1954-10-31,0.02585
5,1954-11-30,0.02591
6,1954-12-31,0.02605


### Tasa de cambio

In [17]:
trm <- read_excel(path, sheet='TRM',skip=1, col_names=c('Date','trm'))
trm <- as.data.frame(trm)
head(trm)

,Date,trm
,<dttm>,<dbl>
1,1991-11-27,693.32
2,1991-11-28,693.99
3,1991-11-29,694.70
4,1991-11-30,694.70
5,1991-12-01,643.42
6,1991-12-02,643.42


## Simulaciones de proyección